In [ ]:
import json
import math
import sys
from collections import deque

def iter_code_accuracy(obj):
    """
    Recursively traverse any JSON-like structure and yield (code, accuracy)
    for objects that contain both keys.
    """
    q = deque([obj])
    while q:
        cur = q.popleft()
        if isinstance(cur, dict):
            # If it looks like a code-accuracy record, yield it
            if "code" in cur and "accuracy" in cur:
                code = cur["code"]
                acc = cur["accuracy"]
                # Be defensive about types
                try:
                    acc = float(acc)
                except (TypeError, ValueError):
                    acc = None
                if isinstance(code, str) and acc is not None:
                    yield code, acc
            # Keep traversing
            for v in cur.values():
                q.append(v)
        elif isinstance(cur, list):
            q.extend(cur)

def main(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    pairs = list(iter_code_accuracy(data))
    if not pairs:
        print("")  # nothing found; print empty string
        return
    # Find top accuracy
    top_acc = max(acc for _, acc in pairs)

    # Compute 1% band around top accuracy (relative to |top_acc|)
    tol = 0.001 * abs(top_acc)
    # If top_acc is exactly 0, only exact matches qualify
    def within_one_percent(acc):
        return abs(acc - top_acc) <= tol

    candidates = [(code, acc) for code, acc in pairs if within_one_percent(acc)]

    # Among candidates, choose the shortest code (by character length).
    # If thereâ€™s a tie, prefer the one with the higher accuracy, then the earlier one.
    candidates.sort(key=lambda ca: (len(ca[0]), -ca[1]))
    best_code = candidates[0][0]
    best_accuracy = candidates[0][1]
    # Print only the code, as requested
    return best_code, best_accuracy, len(pairs)



In [ ]:
filename = f'....json'
best_code, best_accuracy, code_generated = main(filename)